In [1]:
# importing some of the necessary packages
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as plt

In [7]:
# loading the data into a Data Frame
soil_df = pd.read_csv('soil_measures.csv')

# going through some EDA
print(soil_df.head())
print(soil_df.info())
print("\n\n", soil_df.describe().round(2))

    N   P   K        ph  crop
0  90  42  43  6.502985  rice
1  85  58  41  7.038096  rice
2  60  55  44  7.840207  rice
3  74  35  40  6.980401  rice
4  78  42  42  7.628473  rice
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   N       2200 non-null   int64  
 1   P       2200 non-null   int64  
 2   K       2200 non-null   int64  
 3   ph      2200 non-null   float64
 4   crop    2200 non-null   object 
dtypes: float64(1), int64(3), object(1)
memory usage: 86.1+ KB
None


              N        P        K       ph
count  2200.00  2200.00  2200.00  2200.00
mean     50.55    53.36    48.15     6.47
std      36.92    32.99    50.65     0.77
min       0.00     5.00     5.00     3.50
25%      21.00    28.00    20.00     5.97
50%      37.00    51.00    32.00     6.43
75%      84.25    68.00    49.00     6.92
max     140.00   145.00   205.00     9.94


In [9]:
# check for missing data
print(soil_df.isnull().sum())

N       0
P       0
K       0
ph      0
crop    0
dtype: int64
N       0
P       0
K       0
ph      0
crop    0
dtype: int64
